In [1]:
from tensorflow.keras import Input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import cv2 as cv 
import numpy as np

In [2]:
def get_data(path) :
  train_datagen = ImageDataGenerator(rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      validation_split=0.2)

  train_data=train_datagen.flow_from_directory(
      path,
      target_size=(128,128),
      batch_size=64,
      class_mode='categorical',
      shuffle=True,
      subset='training')
  test_data=train_datagen.flow_from_directory(
      path,
      target_size=(128,128),
      batch_size=64,
      class_mode='categorical',
      subset='validation')
  return train_data,test_data

path = '/Users/quangnguyen/Library/Mobile Documents/com~apple~CloudDocs/DS-DungLai/Deep Learning/Money classification/money_data'
train_data,test_data = get_data(path)

Found 3204 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [3]:
def get_model() :
  vgg=VGG16(include_top=False, weights='imagenet')

  # free pre_trained layers,only train dense layer (FC)
  for layer in vgg.layers :
    layer.trainable=False

  # input and output for model
  input = Input(shape=(128,128,3),name='image_size')
  output_vgg = vgg(input)
  x = Flatten(name='flatten')(output_vgg)
  x = Dense(4096,'relu',name='fc1')(x)
  x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
  x = Dense(4096,'relu',name='fc2')(x)
  x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
  x = Dense(4,'softmax',name='softmax')(x)

  vgg_model = Model(inputs=input,outputs=x)
  vgg_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
  return vgg_model
model = get_model() 
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_size (InputLayer)      [(None, 128, 128, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              33558528  
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              167813

In [4]:
model.load_weights('/Users/quangnguyen/Library/Mobile Documents/com~apple~CloudDocs/DS-DungLai/Deep Learning/Money classification/weights.hdf5')

In [5]:
import cv2 
class_name= ['KHONG CO GI','10000','20000','50000']
cap=cv2.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, image_org = cap.read()
    if not ret:
        continue
    image_org = cv2.resize(image_org, dsize=None,fx=0.5,fy=0.5)
    # Resize
    image = image_org.copy()
    image = cv2.resize(image, dsize=(128, 128))
    image = image.astype('float')*1./255
    # Convert to tensor
    image = np.expand_dims(image, axis=0)
    #predict
    predict=model.predict(image)
    print('This picture is :',class_name[y_hat],np.argmax(predict[0]))
    
    if (np.max(predict)>=0.8) and (np.argmax(predict[0])!=0):


        # Show image
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (50, 50)
        fontScale = 1.5
        color = (0, 255, 0)
        thickness = 2

        cv2.putText(image_org, class_name[np.argmax(predict)], org, font,
                    fontScale, color, thickness, cv2.LINE_AA)

    cv2.imshow("Picture", image_org)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This picture is : 50000 3
This p